# Hearthstone Deck Recommendation
### Objetivo: Construir un mazo ganador en contra de un mazo oponente específico para una partida de Hearthstone.

## Librerias

In [50]:
import pandas as pd
import numpy as np

## Base de datos

In [51]:
df = pd.read_csv("data\data.csv")
df.head()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
0,9740,2016-02-19,Unknown,Priest,W,433004,Explorers,Tavern Brawl,1,Reno Priest,...,374,2280,2511,2555,2566,2582,2683,2736,2568,2883
1,9840,2016-02-19,Unknown,Warrior,W,433003,Explorers,Ranked Deck,1,RoosterWarrior,...,1781,1781,2021,2021,2064,2064,2078,2510,2729,2736
2,2600,2016-02-19,Unknown,Mage,W,433002,Explorers,Theorycraft,1,Annoying,...,1793,1801,1801,2037,2037,2064,2064,2078,38710,38710
3,15600,2016-02-19,Unknown,Warrior,W,433001,Explorers,NaN,0,Standart pay to win warrior,...,1657,1721,2018,2296,2262,336,2729,2729,2736,2760
4,7700,2016-02-19,Unknown,Paladin,W,432997,Explorers,Ranked Deck,1,Palamix,...,2027,2029,2029,2064,2078,374,2717,2717,2889,2889


## Cartas y mazos
Definimos el conjunto *cards* con las *N* cartas disponibles para elegir y armar un mazo de tamaño *D*. Las cartas disponibles serán solo las que están enumeradas en la base de datos.

CAMBIOS:
- redefinir N por algo más pequeño y cartas que se pueden elegir
- redefinir deck para repetir cartas

In [52]:
cards_cols = df.iloc[:, 11:41] # cartas en la base de datos
cards = sorted(pd.unique(cards_cols.values.ravel()).tolist()) # conjunto de cartas para elegir

N = len(cards) # tamaño conjunto cartas para elegir
D = 30 # tamaño mazo Hearthstone

In [53]:
N = 40 # N más pequeño para implementar
D = 10 # D más pequeño para implementar

El mazo lo definimos como un vector binario *deck* con *N* entradas y exactamente *D* iguales a 1. Para la construción de un mazo ganador necesitaremos el mazo inicial *deck_p* del jugador, que tenemos el objetivo de mejorar, y el mazo del oponente *deck_o*, que buscamos derrotar en una partida. A continación se presenta un ejemplo de mazos.

In [54]:
deck_p = np.zeros(N, dtype=int) # definir mazo jugador
deck_p[np.random.choice(N, D, replace=False)] = 1 # generar mazo aleatorio jugador
print(f'Mazo jugador: {deck_p}')

deck_o = np.zeros(N, dtype=int) # definir mazo oponente
deck_o[np.random.choice(N, D, replace=False)] = 1 # generar mazo aleatorio oponente
print(f'Mazo oponente: {deck_o}')

Mazo jugador: [0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 0 1]
Mazo oponente: [0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0
 1 0 0]


## Transiciones y acciones
El proceso para mejorar el mazo *deck_p* consta de aplicar múltiples veces la función de transición *transition* determinada por una acción. Una acción *action* es una tupla con las cartas del reemplazo que modifica exactamente una carta del mazo por una carta que actualmente no está incluida. A continuación, un ejemplo del uso de la función *transition*, en donde se elige de forma aleatoria la acción.

In [55]:
def transition(deck, action):
    deck[action[0]] = 0 # eliminamos la carta antigua del mazo
    deck[action[1]] = 1 # agregamos la carta nueva al mazo
    return deck

In [56]:
print(f'Mazo antiguo: {deck_p}')

old_card = np.random.choice(np.where(deck_p == 1)[0]) # elegimos aleatoriamente una carta que está en el mazo
new_card = np.random.choice(np.where(deck_p == 0)[0]) # elegimos aleatoriamente una carta que no está en el mazo
action = (old_card, new_card) # definimos la acción
print(f'Acción: ({action[0]},{action[1]})')

new_deck_p = transition(deck_p, action)
print(f'Mazo nuevo: {new_deck_p}')

Mazo antiguo: [0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 0 1]
Acción: (39,7)
Mazo nuevo: [0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 0 0]


## Win Rate
Para elegir las acciones que nos lleven a encontrar un mazo ganador usaremos el *win_rate* de este mazo. Entonces, para elegir la próxima acción buscaremos cual es la que maximiza el *win_rate* del mazo.

In [ ]:
import keras
import pickle
model = keras.models.load_model("win_rate_NN.keras")
model.summary()
with open("encoder", "rb") as f:
    encoder = pickle.load(f)
with open("win_rate_RF.pkl", "rb") as f:
    rf = pickle.load(f)

In [ ]:
def win_rate_NN(deck,encoder = encoder, model =model):
  mazo_gen_num = deck.drop(columns=['date', 'deck_archetype', 'deck_class', 'deck_format',
       'deck_id', 'deck_set', 'deck_type', 'rating', 'title', 'user']).iloc[15].to_frame().transpose().reset_index()
 # return mazo_gen_num
  hero = deck["deck_class"].iloc[15]
  mazo_gen_cat = pd.DataFrame([{"hero":hero}])
  encoded_categorical = encoder.transform(mazo_gen_cat)
  # # # Step 3: Convert the encoded data back to a DataFrame
  encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out())
  # # Step 4: Concatenate the numerical column with the one-hot encoded categorical columns
  final_df = pd.concat([mazo_gen_num, encoded_df], axis=1).drop(columns=["index"]).rename(columns={"craft_cost":"dust"})
  results = model.predict(final_df)
  return results

## Implementación
Se podrán realizar máximo *D* acciones, ya que a lo más necesitamos modificar completamente el mazo inicial para encontrar un óptimo